In [ ]:
import _misc
import os
import modelPanelSetup as aPanel
import _myLogging

USERID=''
FOLDERGUESS=''
CONNECTIONTYPE=2
MYLOGGER = _myLogging.get_logger("PanelVersion")
TOOLNAME="Reinsurance Strategy Analysis"

dict_userPaths=_misc.configparser_to_dict("config.ini")['dict_userPaths']

def get_panel():
    USERID=os.getlogin()

    try:
        FOLDERGUESS=dict_userPaths[USERID]
    except:
        FOLDERGUESS=None   

    return aPanel.Panel(CONNECTIONTYPE,TOOLNAME,FOLDERGUESS,False) 

thisPanel=get_panel()
thisPanel.view()

In [30]:
import altair as alt
import panel as pn
pn.extension('vega')
alt.data_transformers.enable("vegafusion")

kpi_results_df = thisPanel.analysis.getResults("All","KPI Results").to_pandas()
display(kpi_results_df.head(5))

BokehModel(combine_events=True, render_bundle={'docs_json': {'92399eb2-4dee-4afd-9cb1-242c43d4238d': {'version…

2024-05-30 16:35:54,922 — Analysis — DEBUG — Enter getResults:KPI Results All
2024-05-30 16:35:54,929 — Functions — DEBUG — Start GetOrCreateDataFrameIfNotExists
2024-05-30 16:35:54,936 — Analysis — DEBUG — Exit getResults:KPI Results All


,Scenario,Strategy,Segmentation,Segment,KPI Metric,Percentile,Value,Rank
0,Mitsui,Option 1A,All,All,Gross Loss,NaN,4.245966e+06,9
1,Mitsui,Option 2A,All,All,Gross Loss,NaN,4.245966e+06,3
2,Mitsui,Option 2B,All,All,Gross Loss,NaN,4.245966e+06,1
3,Mitsui,Option 1B,All,All,Gross Loss,NaN,4.245966e+06,8
4,Mitsui,Option 2C,All,All,Gross Loss,NaN,4.245966e+06,5


In [31]:
import importlib
import _chartClasses 
import pandas as pd
import matplotlib
matplotlib.use('agg')
importlib.reload(_chartClasses)
_chartClasses.CreateChart(source_df=kpi_results_df,categorycols=['Scenario','Strategy','Segmentation','Segment','KPI Metric','Return Period'],
            chart_type='Heatmap',
            dffiltercols=['Scenario','Segmentation','Segment'],
            legendcols=['Strategy','KPI Metric','Return Period'],
            valuecols=['Value','Rank'],
            #x_valueCols=['Rank'],
            colCombinations={'KPI Metric':['KPI Metric','Return Period']},
            groupByLegendColumn=False).view.show()

BokehModel(combine_events=True, render_bundle={'docs_json': {'5fd78b5d-6911-4314-b7b4-58d74518dfab': {'version…

result_x
(52, 4)
result_y
(0, 0)
result_z
(0, 0)
2
USER CHART TYPE UPDATED
Column
_y0  None
_y2  None
data shape (52, 4)
result_x
(52, 4)
result_y
(0, 0)
result_z
(0, 0)
2
USER CHART TYPE UPDATED
Column
_y0  None
_y2  None
data shape (52, 4)
USER CHART TYPE UPDATED
Column
_y0  None
_y2  None
Launching server at http://localhost:62417


In [28]:
import altair as alt
import panel as pn
import pandas as pd
import mercury as mr
df = thisPanel.analysis.getResults("All","KPI Results").to_pandas().fillna(0)
df = df[(df['Segmentation'] == 'All') & (df['Segment'] == 'All')]
print(df.shape)
df.head(5)

2024-05-30 16:35:01,852 — Analysis — DEBUG — Enter getResults:KPI Results All
2024-05-30 16:35:01,865 — Functions — DEBUG — Start GetOrCreateDataFrameIfNotExists
2024-05-30 16:35:01,882 — Analysis — DEBUG — Exit getResults:KPI Results All
(234, 8)


,Scenario,Strategy,Segmentation,Segment,KPI Metric,Percentile,Value,Rank
0,Mitsui,Option 1A,All,All,Gross Loss,0.0,4.245966e+06,9
1,Mitsui,Option 2A,All,All,Gross Loss,0.0,4.245966e+06,3
2,Mitsui,Option 2B,All,All,Gross Loss,0.0,4.245966e+06,1
3,Mitsui,Option 1B,All,All,Gross Loss,0.0,4.245966e+06,8
4,Mitsui,Option 2C,All,All,Gross Loss,0.0,4.245966e+06,5


In [ ]:
#WORKING VERSION OF ALTAIR HEATMAP + BAR AND BUMP CHARTS
def get_corr_df(df):
    data = pd.DataFrame()
    for strategy in sorted(df['Strategy'].unique().tolist()):
        _df = df[df['Strategy'] == strategy]
        data[strategy] = _df['Value'].tolist()
    return data.corr().stack().reset_index().rename(columns={0: 'correlation'})

select_x = alt.selection_point(fields=['level_0'], name='select_x', value='Option 1A')
select_y = alt.selection_point(fields=['level_1'], name='select_y', value='Option 2A')

heatmap = alt.Chart(
    get_corr_df(df),
    title='Click a tile to compare timeseries',
    height=250,
    width=250,
).mark_rect().encode(
    alt.X('level_0').title(None),
    alt.Y('level_1').title(None),
    alt.Color('correlation').scale(domain=[0.999, 1], scheme='rainbow'),
    opacity=alt.condition(select_x & select_y, alt.value(1), alt.value(0.4))
).add_params(
    select_x, select_y
)


#Bar chart comparing the two strategies by selected metrics
base = alt.Chart(
    df[['Strategy','KPI Metric','Value','Rank']],
    height=100,
    width=300,
    title='Bar Chart by Metric'
)

bar = base.transform_filter(
    'indexof(datum.Strategy, select_x.level_0) !== -1'
   '| indexof(datum.Strategy, select_y.level_1) !== -1'
).mark_bar().encode(
    alt.X('KPI Metric:N', axis=alt.Axis(labelAngle=-45)).title(None),
    alt.Y('Value:Q'),
    alt.Color('Strategy:N').legend(orient='top').scale(scheme='paired')
)

bump = base.transform_filter(
    'indexof(datum.Strategy, select_x.level_0) !== -1'
   '| indexof(datum.Strategy, select_y.level_1) !== -1'
).mark_line(point=True).encode(
    alt.X('KPI Metric:N', axis=alt.Axis(labelAngle=-45)),
    alt.Y('Rank:O', axis=alt.Axis(grid=True)),
    color = alt.Color("Strategy:N")
).transform_window(
    rank='rank()',
    sort=[alt.SortField("Rank", order="descending")],
    groupby=['KPI Metric:N']
).properties(
    width = 300,
    height = 100,
    title="Bump Chart for Strategies"
)
output = (bar & bump) | heatmap
output
# vega_spec = output.to_dict(format="vega")
# pn.pane.Vega(output.to_dict(format='vega'))

In [ ]:
import pandas as pd
import numpy as np
import altair as alt


# Create timeseries data
rng = np.random.default_rng(905)
ex_ts = pd.DataFrame(
    rng.random((10, 4)),
    columns=['a', 'b', 'c', 'd'],
).assign(
    date=pd.date_range(
        start=pd.to_datetime('2022-02-22')-pd.Timedelta(9, unit='D'),
        end=pd.to_datetime('2022-02-22')).strftime('%Y-%m-%d'),
)

# Create heatmap with selection
select_x = alt.selection_point(fields=['level_0'], name='select_x', value='b')
select_y = alt.selection_point(fields=['level_1'], name='select_y', value='d')
heatmap = alt.Chart(
    ex_ts.drop(columns='date').corr().stack().reset_index().rename(columns={0: 'correlation'}),
    title='Click a tile to compare timeseries',
    height=250,
    width=250,
).mark_rect().encode(
    alt.X('level_0').title(None),
    alt.Y('level_1').title(None),
    alt.Color('correlation').scale(domain=[-1, 1], scheme='blueorange'),
    opacity=alt.condition(select_x & select_y, alt.value(1), alt.value(0.4))
).add_params(
    select_x, select_y
)

# Create chart with individual lines/timeseries
base = alt.Chart(
    ex_ts.melt(
        id_vars='date',
        var_name='category',
        value_name='value',
    ),
    height=100,
    width=300,
    title='Individual timeseries',
)
lines = base.transform_filter(
    # If the category is not in the selected values, the returned index is -1
    'indexof(datum.category, select_x.level_0) !== -1'
   '| indexof(datum.category, select_y.level_1) !== -1'
).mark_line().encode(
    alt.X('date:T').axis(labels=False).title(None),
    alt.Y('value').scale(domain=(0, 1)),
    alt.Color('category').legend(orient='top', offset=-20).title(None)
)

# Create chart with difference between lines/timeseries
dynamic_title = alt.Title(alt.expr(f'"Difference " + {select_x.name}.level_0 + " - " + {select_y.name}.level_1'))
# We pivot transform to get each category as a column
lines_diff = base.transform_pivot(
    'category', 'value', groupby=['date']
# In the calculate transform we use the values from the selection to subset the columns to substract
).transform_calculate(
    difference = f'datum[{select_x.name}.level_0] - datum[{select_y.name}.level_1]'
).mark_line(color='grey').encode(
    alt.X('date:T').axis(format='%Y-%m-%d').title(None),
    alt.Y('difference:Q').scale(domain=(-1, 1)),
).properties(
    title=dynamic_title
)

# Layout the charts
(lines & lines_diff) | heatmap

In [21]:
#1-dimenionsal example
import importlib
import _chartClasses
importlib.reload(_misc)
importlib.reload(_chartClasses)
_chartClasses.CreateChart(source_df=df,
            chart_type='Table-1dim',
            categorycols=['Scenario','Strategy','Segmentation','Segment','KPI Metric','Percentile'],
            dffiltercols=['Scenario','Segmentation','Segment'],
            legendcols=['Strategy','KPI Metric','Percentile'],
            valuecols=['Value','Rank'],
            x_valueCols=['Rank'],
            colCombinations={'KPI Metric':['KPI Metric','Percentile']},
            groupByLegendColumn=True).view.show()

BokehModel(combine_events=True, render_bundle={'docs_json': {'8df56bc7-02b6-43f6-8be9-fc919d321402': {'version…

result_x
(234, 4)
result_y
(0, 0)
result_z
(0, 0)
2
USER CHART TYPE UPDATED
Column
data shape (234, 4)
USER CHART TYPE UPDATED
Column
Launching server at http://localhost:62225


In [ ]:
#1-dimenionsal example
import importlib
import _chartClasses
importlib.reload(_misc)
importlib.reload(_chartClasses)
_chartClasses.CreateChart(source_df=df,
            chart_type='Waterfall',
            categorycols=['Scenario','Strategy','Segmentation','Segment','KPI Metric','Percentile'],
            dffiltercols=['Scenario','Segmentation','Segment','Strategy'],
            legendcols=['KPI Metric'],
            valuecols=['Value'],
            groupByLegendColumn=True).view.show()

In [29]:
#1-dimenionsal example
import importlib
import _chartClasses
importlib.reload(_misc)
importlib.reload(_chartClasses)
_chartClasses.CreateChart(source_df=df,
            chart_type='Heatmap',
            categorycols=['Scenario','Strategy','Segmentation','Segment'],
            dffiltercols=['Scenario','Segmentation','Segment'],
            legendcols=['Strategy','KPI Metric'],
            valuecols=['KPI Metric','Value','Rank'],
            groupByLegendColumn=False).view.show()

BokehModel(combine_events=True, render_bundle={'docs_json': {'a9c199d8-82ee-4b76-bee7-dc8be40ce313': {'version…

result_x
(52, 5)
result_y
(0, 0)
result_z
(0, 0)
2
USER CHART TYPE UPDATED
Column
_y0  None
_y2  None
data shape (52, 5)


ValueError: Cannot display a pandas.DataFrame with duplicate column names.

In [23]:
#2-dimensional example
import importlib
import _chartClasses
importlib.reload(_misc)
importlib.reload(_chartClasses)
_chartClasses.CreateChart(source_df=df,
            chart_type='Radial Chart-2dim', #Chart type (some are 1-dimensional, some are 2-dimensional)
            categorycols=['Scenario','Strategy','Segmentation','Segment'], #Categorical columns
            dffiltercols=['Scenario','Segmentation','Segment'], #Columns to filter the data
            legendcols=['Strategy'], #Column values to be multichoice select
            valuecols=['Value'], #Value column in the dataset
            x_filterCols={'KPI Metric':['Ceded UW Profit','Ceded Premium']}, #For 2-dimensional data, widget to filter x-axis
            x_valueCols=['Value'], #Value column that x_filterCols filters down
            y_filterCols={'KPI Metric':['Net UW Profit','Net Loss']}, #For 2-dimensional data, widget to filter y-axis
            y_valueCols=['Value'], #Value column that y_filterCols filters down
            groupByLegendColumn=True #Whether or not data is grouped by legend column
            ).view.show()



BokehModel(combine_events=True, render_bundle={'docs_json': {'0a0ca51e-173c-49c0-85bc-53117388a673': {'version…

result_x
(9, 2)
self._y1:  Net UW Profit
result_y
(9, 2)
result_z
(0, 0)
1
USER CHART TYPE UPDATED
Column
data shape (9, 3)
USER CHART TYPE UPDATED
Column
Launching server at http://localhost:62254


In [22]:
#3-dimensional example
import importlib
import _chartClasses
importlib.reload(_misc)
importlib.reload(_chartClasses)
_chartClasses.CreateChart(source_df=df,
            chart_type='Table-3dim', #Chart type (some are 1-dimensional, some are 2-dimensional)
            categorycols=['Scenario','Strategy','Segmentation','Segment'], #Categorical columns
            dffiltercols=['Scenario','Segmentation','Segment'], #Columns to filter the data
            legendcols=['Strategy'], #Column values to be multichoice select
            valuecols=['Value'], #Value column in the dataset
            x_filterCols={'KPI Metric':['Ceded UW Profit','Ceded Premium']}, #For 2-dimensional data, widget to filter x-axis
            x_valueCols=['Value'], #Value column that x_filterCols filters down
            y_filterCols={'KPI Metric':['Net UW Profit','Net Loss']}, #For 2-dimensional data, widget to filter y-axis
            y_valueCols=['Value'], #Value column that y_filterCols filters down
            z_filterCols={'KPI Metric':['Net Loss']}, #For 3-dimensional data, widget to filter z-axis
            z_valueCols=['Value'],
            groupByLegendColumn=True #Whether or not data is grouped by legend column
            ).view.show()



BokehModel(combine_events=True, render_bundle={'docs_json': {'95463eae-78e1-4b85-8af6-e6bcefc52964': {'version…

result_x
(9, 2)
self._y1:  Net UW Profit
result_y
(9, 2)
result_z
(9, 2)
USER CHART TYPE UPDATED
Column
data shape (9, 4)
USER CHART TYPE UPDATED
Column
Launching server at http://localhost:62241


In [ ]:
#2-dimensional example
import importlib
import _chartClasses
importlib.reload(_misc)
importlib.reload(_chartClasses)
_chartClasses.CreateChart(source_df=df,
            chart_type='Table-2dim', #Chart type (some are 1-dimensional, some are 2-dimensional)
            categorycols=['Scenario','Strategy','Segmentation','Segment'], #Categorical columns
            dffiltercols=['Scenario','Segmentation','Segment'], #Columns to filter the data
            legendcols=['Strategy'], #Column values to be multichoice select
            valuecols=['Value'], #Value column in the dataset
            x_filterCols={'KPI Metric':['Ceded UW Profit','Ceded Premium']}, #For 2-dimensional data, widget to filter x-axis
            x_valueCols=['Value'], #Value column that x_filterCols filters down
            y_filterCols={'KPI Metric':['Net UW Profit','Net Loss']}, #For 2-dimensional data, widget to filter y-axis
            y_valueCols=['Value'], #Value column that y_filterCols filters down
            # z_filterCols={'KPI Metric':['Net Loss']}, #For 3-dimensional data, widget to filter z-axis
            # z_valueCols=['Value'],
            groupByLegendColumn=True #Whether or not data is grouped by legend column
            ).view.show()

